In [1]:
import pandas as pd
import csv
import pandas.io.gbq as bq

In [12]:
# first find the geartypes
'''
Overall	 
2	Registered	 
3	REGISTERED	 
4	OVERALL	 
5	BetweenPP	 
6	overall
'''

q = '''SELECT c.mmsi mmsi, 
if(a.Length_Type  = 'Overall' or a.Length_Type = 'OVERALL' or a.Length_Type = 'overall', a.Length,null)  length,
a.Length_type length_type,
if(a.Tonnage_Type = 'GRT', Null, a.Tonnage )  tonnage,
a.Tonnage_Type tonnage_type,
a.Engine_Power Engine_Power,
a.Power_Units Power_Units
FROM [world-fishing-827:Registry_matching_sources.WCPFC_20170123_download] a 
inner join [world-fishing-827:Registry_matching_sources.CLAV_12_14_2015] b
on a.VID = b.rfmo_id 
left join [world-fishing-827:CLAV_match_results.v7_results] c
on b.clav_TUVI = c.clav_TUVI
where b.rfmo_name = 'WCPFC'
and c.mmsi is not null
'''

wcpfc = pd.io.gbq.read_gbq(q, project_id='world-fishing-827', verbose = False )

In [13]:
wcpfc = wcpfc.set_index('mmsi')
wcpfc.head()

,length,length_type,tonnage,tonnage_type,Engine_Power,Power_Units
mmsi,,,,,,
412687180,None,Registered,115,GT,237,KW
416004299,23.9,Overall,99.7,GT,940,HP
412689850,None,Registered,128,GT,407,KW
416003700,56.9,Overall,720,GT,1400,HP
416003700,56.9,Overall,720,GT,1400,HP


In [15]:
# 1 ps = 0.735499 kw 
# 1 hp = 0.7457 kw

def calc_kw(a,b):
    try:
        c = 1
        if b.upper() == "HP":
            c = .75
        if b.upper() == "PS":
            c = .74
        return float(a)*c
    except:
        return None


wcpfc['power'] = wcpfc.apply(lambda row: calc_kw(row['Engine_Power'], row['Power_Units']), axis=1)

In [16]:
wcpfc.head()

,length,length_type,tonnage,tonnage_type,Engine_Power,Power_Units,power
mmsi,,,,,,,
412687180,None,Registered,115,GT,237,KW,237.0
416004299,23.9,Overall,99.7,GT,940,HP,705.0
412689850,None,Registered,128,GT,407,KW,407.0
416003700,56.9,Overall,720,GT,1400,HP,1050.0
416003700,56.9,Overall,720,GT,1400,HP,1050.0


In [17]:
wcpfc.to_csv('input_lists/wcpfc.csv')